<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#크롤링할-사이트" data-toc-modified-id="크롤링할-사이트-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>크롤링할 사이트</a></span></li><li><span><a href="#데이터-스키마" data-toc-modified-id="데이터-스키마-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>데이터 스키마</a></span></li><li><span><a href="#Lets_study" data-toc-modified-id="Lets_study-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Lets_study</a></span></li><li><span><a href="#okky" data-toc-modified-id="okky-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>okky</a></span></li></ul></div>

## 크롤링할 사이트
* 네이버 카페: 
    * [독취사](http://cafe.naver.com/dokchi)
        * 스터디>기타: 검색어 '프로그래밍'
    * 안해 ㅅㅂ
* 스터디 사이트:
    * [스터디하자](http://lets-study.co.kr/): 프로그래밍 카테고리
    * 완료
* 커뮤니티:
    * [OKKY](https://okky.kr/): 
        * 커뮤티니> 정기모임/스터디
    * 완료
        
## 데이터 스키마
* id
* home: 크롤링한 사이트
* title: 제목
* content: 본문
* hash_tags: 본문 빈출 단어 top 1-5(본문 길이에 따라 생성)

In [25]:
# libraries
from selenium.webdriver.support.ui import Select
from selenium import webdriver
import pandas as pd
import requests
import json
import time
import bs4
import re
import time

## Lets_study

In [61]:
driver = webdriver.Chrome('chromedriver')  # start webdriver

In [80]:
lets_study_dict = {
    'id': [],
    'title': [],
    'content': [],
    'date': []
}
for page_idx in range(1,4):
    # 스터디 페이지 입력
    lets_study_urls = f'http://lets-study.co.kr/studies?categoryid=4&pageno={page_idx}'
    driver.get(lets_study_urls)
    # 스터디 포스트 클릭
    # 페이지내 모든 링크에 대해서
    n_study_posts_links = len(driver.find_elements_by_css_selector('tr'))
    for link_idx in range(n_study_posts_links):
        study_posts_links = driver.find_elements_by_css_selector('tr') # 링크 가져오기
        study_posts_links[link_idx].click() # 링크 클릭
        current_url = driver.current_url  # 현재 url 가져오기
        current_res = getRequest(current_url)  # 현재 url request
        
        # dict에 저장
        lets_study_dict['id'].append(current_res.select('div.inst>.c-g')[0].text)
        lets_study_dict['title'].append(current_res.select('title')[0].text)
        lets_study_dict['content'].append(current_res.select('.introduce')[0].text)
        lets_study_dict['date'].append(current_res.select('div.inst>.c-l-g')[0].text)

        driver.back() # 뒤로가기

In [86]:
# 저장
lets_study_df = pd.DataFrame.from_dict (lets_study_dict)
lets_study_df.to_csv('lets_study.csv', index=False, mode='w')

## okky

In [160]:
driver = webdriver.Chrome('chromedriver')  # start webdriver

In [179]:
okky_dict = {
    'id': [],
    'title': [],
    'content': [],
    'date': [],
    'tags': [],
}
for page_idx in range(0,120,20):
    okky_url = f'https://okky.kr/articles/gathering?offset={page_idx}&max=20&sort=id&order=desc'
    driver.get(okky_url)
    n_post_a_tag = len(driver.find_elements_by_css_selector('div.clearfix>h5>a'))

    for post_idx in range(n_post_a_tag):
        post_a_tag = driver.find_elements_by_css_selector('div.clearfix>h5>a')

        post_a_tag[post_idx].click()
        current_url = driver.current_url  # 현재 url 가져오기
        current_res = getRequest(current_url)  # 현재 url request

        okky_dict['id'].append(current_res.select('div.panel-heading.clearfix .nickname')[0].text) 
        okky_dict['title'].append(current_res.select('h2.panel-title')[0].text)
        okky_dict['content'].append(current_res.select('article.content-text')[0].text)
        okky_dict['date'].append(current_res.select('span.timeago')[0].text) 
        okky_dict['tags'].append(current_res.select('div.content-tags')[0].text)

        driver.back() # 뒤로가기

In [183]:
# 저장
okky_dict_df = pd.DataFrame.from_dict(okky_dict)
okky_dict_df.to_csv('okky.csv', index=False)